>
> <a class="anchor" id="welcome"></a> 
<img src="https://i.ibb.co/C9QvSmC/task3.png" alt="Drawing"  style="width: 600px;"/><br />
> <b> <span style="font-size: 400%"> <span style='font-family:Helvetica'> Bookstore Dataset</b>
>  <br />
> <img src="https://i.pinimg.com/originals/4c/79/70/4c7970c030c4b8a8322171dd6a498cb0.gif" alt="Drawing" style="width: 396px;"/> <br />
> <br />
> <span style='font-family:Verdana '> Msc in Business Analytics @ AEUB  <br />
>  <span style='font-family:Verdana '> Big Data Systems and Architectures - PT  <br />
>  <span style='font-family:Verdana '> Professor: Thanasis Vergoulis <br />
>  <span style='font-family:Verdana '> Student: Despotis Spyridon  <br />
>  <span style='font-family:Verdana '> Student ID: P2822111 <br />

***

> <img src="https://i.ibb.co/0rTP5XM/table-of-contents.png" alt="Drawing" style="width: 480px;"/> <br />
> * [Task1](#question1) <br />
> * [Task2](#question2) <br />
> * [Task3](#question3) <br />
> * [Task4](#question4) <br />
> * [Conclusions for all projects (bonus)](#question5) <br />
***

> <b> <span style="font-size: 200%"> <span style='font-family:Helvetica'>Lets Start our Analysis!</b>

* Importing the Libraries and creating the spark Session

In [1]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml import Pipeline
import pyspark.ml.evaluation
from pyspark.ml.feature import IndexToString, MinMaxScaler, StringIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import isnull, when, count, col
from pyspark.ml.evaluation import RegressionEvaluator
spark = SparkSession.builder.appName("books_5000").getOrCreate()

<a class="anchor" id="question1"></a> 
</b>
<b> <span style="font-size: 200%"> <span style='font-family:Helvetica'>Task 1 "Preparing the feature vectors"</b> 
</b>

* Reading the books_5000.json using spark json read function

In [6]:
books = spark.read.json("books_5000.json")

* Creating new dataframe by slecting the appropriate columns from books dataset

In [7]:
df1 = books.select(col("average_rating"),col("language_code"),col("num_pages"),col("ratings_count"),col("publication_year"))

* Encode string columns "language_code", "publication_yearIndex" to new column with indexes
* Then we drop the unwanted columns

In [8]:
df1 = StringIndexer(inputCol='language_code', outputCol='language_codeIndex').fit(df1).transform(df1)
df1 = StringIndexer(inputCol='publication_year', outputCol='publication_yearIndex').fit(df1).transform(df1)
df3 = df1.drop("language_code","publication_year")
df3.show()

+--------------+---------+-------------+------------------+---------------------+
|average_rating|num_pages|ratings_count|language_codeIndex|publication_yearIndex|
+--------------+---------+-------------+------------------+---------------------+
|          4.12|         |            1|               0.0|                  0.0|
|          3.94|         |           16|               3.0|                  2.0|
|          4.28|      146|           51|               1.0|                  5.0|
|          4.05|         |            6|               1.0|                  0.0|
|          4.06|      272|           51|               6.0|                 20.0|
|          3.44|      206|           46|               0.0|                 12.0|
|          4.15|      224|           39|               1.0|                  2.0|
|          3.16|      160|           38|               0.0|                  2.0|
|          3.51|      160|           44|               0.0|                  2.0|
|          4.00|

* Then we check for the data types of each column

In [9]:
df3.dtypes

[('average_rating', 'string'),
 ('num_pages', 'string'),
 ('ratings_count', 'string'),
 ('language_codeIndex', 'double'),
 ('publication_yearIndex', 'double')]

* Then we proceed by converting each column to the appropriate data type

In [10]:
df3 = df3.withColumn("num_pages", df3["num_pages"].cast('int'))
df3 = df3.withColumn("language_codeIndex", df3["language_codeIndex"].cast('double'))
df3 = df3.withColumn("publication_yearIndex", df3["publication_yearIndex"].cast('double'))
df3 = df3.withColumn("ratings_count", df3["ratings_count"].cast('int'))
df3 = df3.withColumn("average_rating", df3["average_rating"].cast('double'))
df3.show()

+--------------+---------+-------------+------------------+---------------------+
|average_rating|num_pages|ratings_count|language_codeIndex|publication_yearIndex|
+--------------+---------+-------------+------------------+---------------------+
|          4.12|     null|            1|               0.0|                  0.0|
|          3.94|     null|           16|               3.0|                  2.0|
|          4.28|      146|           51|               1.0|                  5.0|
|          4.05|     null|            6|               1.0|                  0.0|
|          4.06|      272|           51|               6.0|                 20.0|
|          3.44|      206|           46|               0.0|                 12.0|
|          4.15|      224|           39|               1.0|                  2.0|
|          3.16|      160|           38|               0.0|                  2.0|
|          3.51|      160|           44|               0.0|                  2.0|
|           4.0|

* Next we proceed by looking for missing values in our dataset

In [11]:
from pyspark.sql.functions import isnull, when, count, col
df3.select([count(when(isnull(c), c)).alias(c) for c in df3.columns]).show()

+--------------+---------+-------------+------------------+---------------------+
|average_rating|num_pages|ratings_count|language_codeIndex|publication_yearIndex|
+--------------+---------+-------------+------------------+---------------------+
|             0|     1382|            0|                 0|                    0|
+--------------+---------+-------------+------------------+---------------------+



* In order to proceed our analysis we drop the empty values
* And we check again if the nulls are dropped

In [12]:
df3 = df3.na.drop()
from pyspark.sql.functions import isnull, when, count, col
df3.select([count(when(isnull(c), c)).alias(c) for c in df3.columns]).show()

+--------------+---------+-------------+------------------+---------------------+
|average_rating|num_pages|ratings_count|language_codeIndex|publication_yearIndex|
+--------------+---------+-------------+------------------+---------------------+
|             0|        0|            0|                 0|                    0|
+--------------+---------+-------------+------------------+---------------------+



* Next we create our column features with vector assembler
* We print the shema

In [13]:
assembler = VectorAssembler(
inputCols=['num_pages',
 'ratings_count',
 'language_codeIndex',
 'publication_yearIndex'],
outputCol="features")
output = assembler.transform(df3)
output.printSchema()

root
 |-- average_rating: double (nullable = true)
 |-- num_pages: integer (nullable = true)
 |-- ratings_count: integer (nullable = true)
 |-- language_codeIndex: double (nullable = false)
 |-- publication_yearIndex: double (nullable = false)
 |-- features: vector (nullable = true)



* We select the two columns for regression alalysis
* we print the shema

In [14]:
output.select("features","average_rating")
final_data = output.select("features","average_rating")
final_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- average_rating: double (nullable = true)



<a class="anchor" id="question2"></a> 
</b>
<b> <span style="font-size: 200%"> <span style='font-family:Helvetica'>Task 2 "Preparing the training and testing datasets"</b> 
</b>

* We split our data to train and test with 70% and 30% of total data available

In [15]:
train_data,test_data = final_data.randomSplit([0.7,0.3], seed =42)

* Finally, we call the linear regression package we imported with the two columns

In [16]:
lr =LinearRegression(featuresCol='features',labelCol='average_rating', regParam=1.0) 

<a class="anchor" id="question3"></a> 
</b>
<b> <span style="font-size: 200%"> <span style='font-family:Helvetica'>Task 3 "Training the model"</b> 
</b>

* We proceed by training our model and printing the coefficients and interecept

In [17]:
lr_model = lr.fit(train_data)
print("Coefficients: {}".format(lr_model.coefficients))
print("Intercept: {}".format(lr_model.intercept))

Coefficients: [0.00023537229167000466,7.084859811602435e-07,0.001715765983396137,0.0006774048088672396]
Intercept: 3.8749788154680296


* Next we summarize the model over the training set and print out some metrics (RMSE, r2)

In [18]:
trainingSummary = lr_model.summary
print("RMSE: {}".format(trainingSummary.rootMeanSquaredError))
print("r2: {}".format(trainingSummary.r2))

RMSE: 0.4281650560028555
r2: 0.023422959229442708


* Next we proceed with testing our model

In [19]:
lr_predictions = lr_model.transform(test_data)
lr_predictions.select("prediction","average_rating","features").show(5)

+------------------+--------------+--------------------+
|        prediction|average_rating|            features|
+------------------+--------------+--------------------+
| 3.880706165072637|          3.49| [14.0,55.0,1.0,1.0]|
|3.9112205916226452|          4.17|[20.0,18.0,1.0,44.0]|
|3.8845937268793334|          4.28| [22.0,16.0,1.0,4.0]|
| 3.888205495925622|           4.0|  [23.0,1.0,1.0,9.0]|
| 3.885548304652759|          4.65| [23.0,75.0,1.0,5.0]|
+------------------+--------------+--------------------+
only showing top 5 rows



<a class="anchor" id="question4"></a> 
</b>
<b> <span style="font-size: 200%"> <span style='font-family:Helvetica'>Task 4 "Evaluating the accuracy of the model"</b> 
</b>

* Next we calculate the r2 on the test data

In [20]:
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="average_rating",metricName="r2")
print("R Squared (R2) on test data = {}".format(lr_evaluator.evaluate(lr_predictions)))

R Squared (R2) on test data = 0.014475935934410322


<a class="anchor" id="question5"></a> 
</b>
<b> <span style="font-size: 200%"> <span style='font-family:Helvetica'>Task 5 "Conclusions for all Spark Projects"</b> 
> <img src="https://i.pinimg.com/originals/bb/37/72/bb3772f42dafd107b673974f80534e6c.gif" alt="Drawing" style="width: 396px;"/> <br />
</b>

* Implementing the tasks in Spark were a great chance to experiment with new open source, distributed computing framework and set of libraries.
* Some benefits of using Spark is simplicity, scalability and compatibility (R, Python pandas, and scikit-learn)
* The tasks were gradually getting harder, so they were well represented for someone with entry level knowledge
* Also i learned deeper how to construct a well structured jupyter notebook
* I riched my portofolio with spark projects that are in demand nowadays

***

[Scroll To Top ^](#welcome) <img style="float: right;" src="https://i.ibb.co/w7Lbd0X/2-AUEB-white-HR.jpg"  alt="aueb" width="550px"/>
